In [4]:
import requests

In [5]:
link = "http://www.rightmove.co.uk/property-to-rent/find.html?minBedrooms=2&sortType=6&viewType=LIST&channel=RENT&index=0&maxPrice=1000&radius=0.0&numberOfPropertiesPerPage=499&locationIdentifier=USERDEFINEDAREA%5E%7B%22polylines%22%3A%22kcmtIfpjRhZ%7ELzXnmBxFlqCmb%40hw%40qgAs%5Euu%40qaA_N_uB%7ESmq%40d%5Ds%7B%40n%60A%3F%22%7D"

page = requests.get(link)

In [6]:
search_file = open("data/rightmovesearch.html", "w")
search_file.write(page.text)
search_file.close()

In [7]:
from bs4 import BeautifulSoup

search_file = open("data/rightmovesearch.html", "r")

soup = BeautifulSoup(search_file, 'html.parser')

In [8]:
import re
id_list = soup.find_all("a", {"class": "propertyCard-anchor"})

for id in id_list:
    print_id = re.sub("prop","",id["id"])
    print(print_id) if len(print_id) ==8 else None

52157724
52627572
52332516
52632474
64105495
52958376
52897554
71332193
52840455
63907516
63907492
51068713
71124878
52589718
52561479
63614986
52367112
52328166
52049370
69936770
69451298
40381836


In [10]:
results_id = "l-searchResults"

results = soup.find("div", {"id": results_id})

In [20]:
result_class = "propertyCard-img-link"

cards = results.find_all("a", {"class": result_class})

In [21]:
for card in cards:
    if card["href"] != "":
        print(card["href"])

/property-to-rent/property-64105495.html
/property-to-rent/property-52958376.html
/property-to-rent/property-52897554.html
/property-to-rent/property-71332193.html
/property-to-rent/property-52840455.html
/property-to-rent/property-63907516.html
/property-to-rent/property-63907492.html
/property-to-rent/property-63907453.html
/property-to-rent/property-51068713.html
/property-to-rent/property-71124878.html
/property-to-rent/property-52632474.html
/property-to-rent/property-52589718.html
/property-to-rent/property-52561479.html
/property-to-rent/property-63614986.html
/property-to-rent/property-52367112.html
/property-to-rent/property-52627572.html
/property-to-rent/property-52332516.html
/property-to-rent/property-52328166.html
/property-to-rent/property-52157724.html
/property-to-rent/property-52049370.html
/property-to-rent/property-69936770.html
/property-to-rent/property-69451298.html
/property-to-rent/property-40381836.html


In [22]:
individual_link = "http://www.rightmove.co.uk" + cards[1]["href"]

individual_page = requests.get(individual_link)

In [23]:
search_file = open("data/rightmoverent.html", "w")
search_file.write(individual_page.text)
search_file.close()

In [24]:
search_file = open("data/rightmoverent.html", "r")

individual_soup = BeautifulSoup(search_file, "html.parser")

In [25]:
import re

price = individual_soup.find("p", {"id":"propertyHeaderPrice"}).getText()
re.findall("\d+", price)

['995']

In [26]:
individual_soup.find("h1", {"itemprop": "name"}).getText()

'2 bedroom flat to rent'

In [49]:
import re

class rightMoveAddress():
    
    def __init__(self, address_string):
        self.raw = re.sub("\s", " ", address_string)
        self.raw = re.sub(" +", " ", self.raw).split(",")
        
        self.postcode = None
        self.street = None
        self.tags = list()
        
    def parse(self):
        for item in self.raw:
            if re.match("[A-Z]{1,2}[0-9R]{1,2}", item[1:]):
                self.postcode = item[1:]
            elif "street" in item.lower():
                self.street = item[1:].title()
            else:
                print(item.lower())
                self.tags.append(item[1:].title())
    
    def to_json(self):
        return {
            "postcode": self.postcode,
            "street": self.street,
            "tags": self.tags
        }
                

for link in cards:
    individual_link = "http://www.rightmove.co.uk" + link["href"]

    individual_page = requests.get(individual_link)
    individual_soup = BeautifulSoup(individual_page.content, "html.parser")

    address = re.sub("\s", " ", individual_soup.find("address", {"itemprop": "address"}).getText())
    address = re.sub(" +"," ",address).split(",")
    print(individual_link)
    print(address)
    address_obj = rightMoveAddress(individual_soup.find("address", {"itemprop": "address"}).getText())
    address_obj.parse()
    print(address_obj.to_json())

http://www.rightmove.co.uk/property-to-rent/property-64105495.html
[' GRINDLAY STREET', ' CITY CENTRE', ' EH3 9AS']
 city centre
{'postcode': 'EH3 9AS', 'street': None, 'tags': ['City Centre']}
http://www.rightmove.co.uk/property-to-rent/property-52958376.html
[' Grassmarket', ' Old Town', ' Edinburgh']
 grassmarket
 old town
 edinburgh
{'postcode': None, 'street': None, 'tags': ['Grassmarket', 'Old Town', 'Edinburgh']}
http://www.rightmove.co.uk/property-to-rent/property-52897554.html
[' William Street', ' Edinburgh', ' ']
 edinburgh
 
{'postcode': None, 'street': 'William Street', 'tags': ['Edinburgh', '']}
http://www.rightmove.co.uk/property-to-rent/property-71332193.html
[' Sciennes', ' Marchmont', ' Edinburgh']
 sciennes
 marchmont
 edinburgh
{'postcode': None, 'street': None, 'tags': ['Sciennes', 'Marchmont', 'Edinburgh']}
http://www.rightmove.co.uk/property-to-rent/property-52840455.html
[' Northumberland Street', ' New Town', ' Edinburgh']
 new town
 edinburgh
{'postcode': None

AttributeError: 'NoneType' object has no attribute 'getText'

In [57]:
key_features = individual_soup.find("div", {"class": "key-features"}).find_all("li")

feature_list = [feature.getText() for feature in key_features]

feature_list

['Double Bedrooms: 2',
 'Central Heating Gas',
 'Carpet Flooring',
 'Council Tax Band E',
 'Smoke Detector',
 'Gas Oven',
 'Gas Hob',
 'Washing Machine',
 'Fridge',
 'Bath']

In [63]:
description = individual_soup.find("p", {"itemprop":"description"}).getText()
description = re.sub("\s", " ", description)
re.sub(" +", " ", description)[1:]

"Centrally located FURNISHED first floor two double bedroom flat located right in the heart of Edinburgh's old town. With windows looking out directly onto Grassmarket which has many excellent pubs, restaurants and shops. The flat is also just a short jaunt up castle hill to get to Princes Street which has bus routes to everywhere in the city, as well as Edinburgh Waverley station.The flat comprises of a comfortable living room with small dining area, Fitted kitchen, two double bedrooms and bathroom with shower, Further benefits include gas central heating, gas oven, and the excellent location.Landlord Registration: 101515/230/27300EPC Rating:C"

In [56]:
info_div =  individual_soup.find("div", {"id": "lettingInformation"})
info_table = info_div.find_all("tr")

info_dict = {}

for row in info_table:
    key_value = row.find_all("td")
    key = key_value[0].getText().split(":")[0]
    value = key_value[1].getText()
    info_dict[key] = value

info_dict

{'Added on Rightmove': '14 February 2018 (4 days ago)',
 'Date available': '06/04/2018',
 'Deposit': '£1195',
 'Furnishing': 'Furnished',
 'Letting type': 'Long term'}

In [82]:
import requests
from bs4 import BeautifulSoup
import re

class address():
    
    def __init__(self, address_string):
        self.raw = re.sub("\s", " ", address_string)
        self.raw = re.sub(" +", " ", self.raw).split(",")
        
        self.postcode = None
        self.street = None
        self.tags = list()
        
    def parse(self):
        for item in self.raw:
            if re.match("[A-Z]{1,2}[0-9R]{1,2}", item[1:]):
                self.postcode = item[1:]
            elif "street" in item.lower():
                self.street = item[1:].title()
            else:
                self.tags.append(item[1:].title())
    
    def to_json(self):
        return {
            "postcode": self.postcode,
            "street": self.street,
            "tags": self.tags
        }
    
class key_info():
    
    def __init__(self, soup):
        
        """
        rent
        date available
        deposit
        letting type
        desposit
        furnished
        epc
        council tax
        
        """
        self.rent = int(re.findall("\d+", soup.find("p", {"id":"propertyHeaderPrice"}).getText())[0])
        self.letting_information = {}

        info_div =  soup.find("div", {"id": "lettingInformation"})
        info_table = info_div.find_all("tr")
        
        for row in info_table:
            key_value = row.find_all("td")
            key = key_value[0].getText().split(":")[0]
            value = key_value[1].getText()
            if key == "Date available":
                self.date_available = value
            elif key == "Deposit":
                self.deposit = int(value[1:])
            elif key == "Furnishing":
                self.furnishing = value
            elif key == "Letting type":
                self.letting_type = value

class rentedAccomodation():
    
    def __init__(self, id):
        self.href = "http://www.rightmove.co.uk/property-to-rent/property-" + str(id) + ".html"
        self.soup = BeautifulSoup(requests.get(self.href).content, "html.parser")
        
    def parse(self):
        self.title = self.soup.find("h1", {"itemprop": "name"}).getText()
        
        self.address = address(self.soup.find("address", {"itemprop": "address"}).getText())
        self.address.parse()
        
        self.key_info = key_info(self.soup)
        
        self.description = re.sub("\s", " ", self.soup.find("p", {"itemprop":"description"}).getText())
        self.description = re.sub(" +", " ", self.description)[1:]
        
        self.tags = [tag.getText() for tag in self.soup.find("div", {"class": "key-features"}).find_all("li")]
        

In [83]:
entry = rentedAccomodation("52157724")
entry.parse()
entry.address.to_json()
entry.key_info.date_available
entry.tags

['Double Bedrooms: 1',
 'Single Bedrooms: 1',
 'Garden Access',
 'Parking: Permit Zone',
 'Central Heating Gas',
 'Council Tax Band D',
 'Bath',
 'Shower',
 'Electric Hob',
 'Electric Oven']

In [ ]:
{
    "title": "", //
    "href": "", //
    "address": 
    {
        "postcode": "", //
        "street": "", //
        "latitude": "", //
        "longitude": "", //
        "city": "", //
        "area": "", //
        "tags": [] //
    },
    "description": "", //
    "key_info": 
    {
        "rent": , //
        "date_available": , //
        "deposit": , //
        "letting_type": , //
        "furnishing": , //
        "epc": , //
        "council_tax": , //
        "running_costs": 
        {
            "insurrance": float, //
            "energy": float, //
            "water": float, //
            "council_tax": float, //
        }
    }
    "tags": []
}

In [7]:
%load_ext autoreload
%autoreload 2

In [3]:
from propertyscraper.accomodation import rentedAccomodation

entry = rentedAccomodation.from_rightmove("52157724")



In [4]:
entry.to_json()

{'address': {'area': None,
  'city': None,
  'lattitude': None,
  'longitude': None,
  'postcode': None,
  'street': None,
  'tags': ['Livingstone Place', 'Sciennes', 'Edinburgh']},
 'description': 'Centrally located FURNISHED two bedroom flat located seconds from the Meadows and close to excellent bus links, shops and the University. This well-proportioned flat comprises of a hallway, good size lounge, fully fitted kitchen, utility room, double bedroom, single bedroom and bathroom with shower. The property further benefits from gas central heating, entry phone access, permit zone parking and shared rear garden. Professionals only, no smokers or pets allowed. Landlord Registration No: 92556/230/03090EPC Rating: C',
 'href': 'http://www.rightmove.co.uk/property-to-rent/property-52157724.html',
 'key_info': {'council_tax': None,
  'date_available': '21/02/2018',
  'deposit': 1095,
  'epc': None,
  'furnishing': 'Furnished',
  'letting_type': 'Long term',
  'rent': 895,
  'running_costs':